<a href="https://colab.research.google.com/github/eccho03/datamining_project_clothes/blob/main/item_base_collaborate_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import re

# 데이터 업로드

In [12]:
size_data = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/apriori_result.csv')
allowed_sizes = ['XL이상', 'L', 'M', 'S', 'XS이하']
size_data = size_data[size_data['size'].isin(allowed_sizes)]

In [13]:
size_data

,min_height,max_height,min_weight,max_weight,size,confidence
0,165.0,169.0,80.0,inf,XL이상,0.993502
1,NaN,NaN,80.0,inf,XL이상,0.988845
2,170.0,174.0,80.0,inf,XL이상,0.985016
3,NaN,NaN,75.0,79.0,XL이상,0.963593
4,160.0,164.0,70.0,74.0,XL이상,0.918229
...,...,...,...,...,...,...
200,170.0,174.0,NaN,NaN,L,0.050168
203,160.0,164.0,NaN,NaN,XS이하,0.049284
204,165.0,169.0,NaN,NaN,XL이상,0.049140
206,NaN,NaN,55.0,59.0,L,0.047696


In [14]:
women = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/products_info_women.csv')
men = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/products_info_men.csv')

In [15]:
def parse_price(price_str):
    return int(re.sub(r'[^0-9]', '', price_str))  # 기존가격 데이터(문자열 -> 숫자)

women['가격'] = women['가격'].apply(parse_price)
men['가격'] = men['가격'].apply(parse_price)

# 정보 입력

In [29]:
from ipywidgets import widgets, interactive, HBox, VBox

def recommand_gender(gender, height, weight, min_price, max_price):
    user_height = height
    user_weight = weight

    print("-" * 50)
    print(f"입력된 정보: 성별 = {gender}, 키 = {height}cm, 몸무게 = {weight}kg")
    print(f"원하는 가격대 = {min_price}~{max_price}원")
    print("-" * 50)

def format_price(value):
    """숫자를 쉼표로 포맷."""
    return f"{value:,}"

# 위젯 생성
gender_widget = widgets.ToggleButtons(
    options=['남', '여'],
    description='성별:',
)

height_widget = widgets.FloatText(
    value=170,
    description='키 (cm):',
)

weight_widget = widgets.FloatText(
    value=60,
    description='몸무게 (kg):',
)

min_price_widget = widgets.IntText(
    value=50000,
    description='선호 가격대:',
)

max_price_widget = widgets.IntText(
    value=150000,
    description='~',
)

# 최소/최대 가격 위젯 가로 배치
price_widget = HBox([min_price_widget, max_price_widget])

# 전체 UI 세로 배치
ui = VBox([gender_widget, height_widget, weight_widget, price_widget])

# interactive로 함수 연결
output = interactive(
    recommand_gender,
    gender=gender_widget,
    height=height_widget,
    weight=weight_widget,
    min_price=min_price_widget,
    max_price=max_price_widget,
)

# UI와 출력 함께 표시
display(ui)

In [30]:
user_gender = 'M' if gender_widget.value == '남' else 'W'

if user_gender == 'W':
    data = women
elif user_gender == 'M':
    data = men
else:
    raise ValueError("성별은 'female' 또는 'male'이어야 합니다.")

In [31]:
data

,page_url,제품 소개,제품명,cate1,cate2,가격,성별,XS이하,S,M,L,XL이상
0,https://www.nike.com/kr/t/조던-남성-크루넥-티셔츠-VLdIyv...,남성 크루넥 티셔츠,조던,탑 & 티셔츠,그래픽 티셔츠,69000,M,0,1,1,1,1
1,https://www.nike.com/kr/t/테이텀-남성-티셔츠-wduHw5h9/...,남성 티셔츠,테이텀,탑 & 티셔츠,그래픽 티셔츠,75000,M,0,1,1,1,1
2,https://www.nike.com/kr/t/조던-mvp-남성-긴팔-티셔츠-zD2...,남성 긴팔 티셔츠,조던 MVP,탑 & 티셔츠,그래픽 티셔츠,65000,M,0,0,0,0,1
3,https://www.nike.com/kr/t/스포츠웨어-남성-오버사이즈-긴팔-티셔...,남성 오버사이즈 긴팔 티셔츠,나이키 스포츠웨어,탑 & 티셔츠,그래픽 티셔츠,75000,M,0,1,1,1,1
4,https://www.nike.com/kr/t/스포츠웨어-프리미엄-에센셜-남성용-오...,남성용 오버사이즈 티셔츠,나이키 스포츠웨어 프리미엄 에센셜,탑 & 티셔츠,그래픽 티셔츠,65000,M,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
392,https://www.nike.com/kr/t/클럽-남성-폴리-니트-트랙수트-xLd...,남성 폴리 니트 트랙수트,나이키 클럽,트랙수트,트랙수트 셋업,109000,M,0,0,0,0,1
393,https://www.nike.com/kr/t/드라이-핏-아카데미-남성-축구-트랙수...,남성 축구 트랙수트,나이키 드라이 핏 아카데미,트랙수트,트랙수트 셋업,95000,M,0,0,1,1,1
394,https://www.nike.com/kr/t/조던-스포츠-잼-남성-웜업-팬츠-VF...,남성 웜업 팬츠,조던 스포츠 잼,트랙수트,트랙수트 셋업,119000,M,1,1,1,1,1
395,https://www.nike.com/kr/t/조던-스포츠-잼-남성-워밍업-재킷-D...,남성 워밍업 재킷,조던 스포츠 잼,트랙수트,트랙수트 셋업,145000,M,0,1,1,1,1


In [27]:
selected_item_index = -1
def recommand_clothes(select_clothes):
    global selected_item_index
    selected_item_index = data[data["제품명"] == select_clothes].index[0]

select_clothes_widget = widgets.Dropdown(
    options=[i for i in data["제품명"].tolist()],
    value = data["제품명"].iloc[0],
    description='원하는 옷: '
)

output = interactive(
    recommand_clothes,
    select_clothes=select_clothes_widget,
)

# UI와 출력 함께 표시
display(output)

interactive(children=(Dropdown(description='원하는 옷: ', options=('조던', '테이텀', '조던 MVP', '나이키 스포츠웨어', '나이키 스포츠웨어 …

In [33]:
##### 사용자 정보 #####
user_height = height_widget.value  # 키
user_weight = weight_widget.value   # 몸무게
min_price = min_price_widget.value    # 최소 가격
max_price = max_price_widget.value    # 최대 가격
select_clothes = selected_item_index # 원하는 옷

##### 입력된 정보를 확인 #####
print("입력된 정보를 확인합니다")
print(f"성별 = {user_gender}, 키 = {user_height}cm, 몸무게 = {user_weight}kg")
print(f"원하는 가격대 = {min_price} ~ {max_price}원")
print(f"원하는 옷 인덱스 = {select_clothes}")

입력된 정보를 확인합니다
성별 = M, 키 = 170.0cm, 몸무게 = 60.0kg
원하는 가격대 = 50000 ~ 150000원
원하는 옷 인덱스 = 0


# 사이즈 도출

In [34]:
# 키와 몸무게로부터 사이즈 도출 함수
def get_size_from_height_weight(height, weight):
    size_rows = size_data[
        ((size_data['min_height'].isna()) | (size_data['min_height'] <= height)) &
        ((size_data['max_height'].isna()) | (size_data['max_height'] >= height)) &
        ((size_data['min_weight'].isna()) | (size_data['min_weight'] <= weight)) &
        ((size_data['max_weight'].isna()) | (size_data['max_weight'] >= weight))
    ]
    if not size_rows.empty:
        # Confidence가 가장 높은 사이즈 선택
        best_size_row = size_rows.loc[size_rows['confidence'].idxmax()]
        return best_size_row['size']
    else:
        return None

In [35]:
user_size = get_size_from_height_weight(user_height, user_weight)

In [36]:
user_size

'XL이상'

# item 기반 협업 필터링

In [37]:
from IPython.display import display

In [38]:
import re
from urllib.parse import quote

# 원본 URL 문자열
url = "https://www.nike.com/kr/t/조던-브루클린-플리스-남성-크루넥-스"

# 한글을 찾는 정규식 패턴
def encode_korean(url):
    return re.sub(r'[\uac00-\ud7a3]+', lambda match: quote(match.group(0)), url)

In [40]:
if user_size is None:
    print("사용자에게 맞는 사이즈가 없습니다.")
else:
    # 사용자가 선택한 아이템의 index를 입력받음
    selected_item_index = select_clothes
    selected_item = data.loc[selected_item_index]

    # 사이즈와 가격 범위 필터링: 사용자의 사이즈와 맞는 아이템 필터링
    filtered_items = data[(data[user_size] == 1) & (data['가격'] >= min_price) & (data['가격'] <= max_price)].copy()

    # 필터링된 데이터 개수 출력
    filtered_count = len(filtered_items)
    print(f"사용자 사이즈 '{user_size}' 및 가격 범위 {min_price}-{max_price}에 맞는 아이템 개수: {filtered_count}")

    # 아이템 리스트에 선택한 아이템 추가
    filtered_items = pd.concat([selected_item.to_frame().T, filtered_items], ignore_index=True)

    # 아이템 속성을 활용하여 아이템 간 유사도 계산 (아이템 기반 협업 필터링)
    df_features = pd.get_dummies(filtered_items[['가격', 'cate1', 'cate2']])
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_features)

    # 코사인 유사도를 사용해 아이템 간 유사도 계산
    item_similarity_matrix = cosine_similarity(scaled_features)

    # 선택한 아이템의 인덱스를 기준으로 유사도 계산
    item_similarities = item_similarity_matrix[filtered_items.index.get_loc(0)]

    # 유사도 기준으로 상위 5개 아이템 추천
    top_n = 5  # 추천할 아이템 개수
    recommended_indices = np.argsort(item_similarities)[::-1][1:top_n+1]  # 자기 자신 제외하고 상위 N개
    recommended_items = filtered_items.iloc[recommended_indices]

    # 추천 결과 출력 (사용자가 선택한 옷이 사이즈가 없는 경우 추천 목록에서 제외)
    recommended_items = recommended_items[(recommended_items[user_size] == 1) & (recommended_items['가격'] >= min_price) & (recommended_items['가격'] <= max_price)]

    pd.set_option('display.max_colwidth', None)
    recommended_items['page_url'] = recommended_items['page_url'].apply(encode_korean)
    print("사용자가 선호한 아이템:")
    display(selected_item[['page_url', '제품명', 'cate1', 'cate2', '가격']])
    print("사용자에게 추천할 아이템들:")
    display(recommended_items[['page_url', '제품명', 'cate1', 'cate2', '가격']])

사용자 사이즈 'XL이상' 및 가격 범위 50000-150000에 맞는 아이템 개수: 234
사용자가 선호한 아이템:


,0
page_url,https://www.nike.com/kr/t/조던-남성-크루넥-티셔츠-VLdIyvtF/HQ2666-045
제품명,조던
cate1,탑 & 티셔츠
cate2,그래픽 티셔츠
가격,69000


사용자에게 추천할 아이템들:


,page_url,제품명,cate1,cate2,가격
15,https://www.nike.com/kr/t/%EC%97%90%EC%96%B4-%EC%A1%B0%EB%8D%98-%EC%9B%8C%EB%93%9C%EB%A7%88%ED%81%AC-85-%EB%82%A8%EC%84%B1-%ED%8B%B0%EC%85%94%EC%B8%A0-Uf95fsbq/HF3447-085,에어 조던 워드마크 85,탑 & 티셔츠,그래픽 티셔츠,69000
11,https://www.nike.com/kr/t/%EC%97%90%EC%96%B4-%EC%A1%B0%EB%8D%98-%EC%9B%8C%EB%93%9C%EB%A7%88%ED%81%AC-85-%EB%82%A8%EC%84%B1-%ED%8B%B0%EC%85%94%EC%B8%A0-ZPdFH7Pa/HF5490-480,에어 조던 워드마크 85,탑 & 티셔츠,그래픽 티셔츠,69000
1,https://www.nike.com/kr/t/%EC%A1%B0%EB%8D%98-%EB%82%A8%EC%84%B1-%ED%81%AC%EB%A3%A8%EB%84%A5-%ED%8B%B0%EC%85%94%EC%B8%A0-VLdIyvtF/HQ2666-045,조던,탑 & 티셔츠,그래픽 티셔츠,69000
51,https://www.nike.com/kr/t/%ED%94%84%EB%9D%BC%EC%9D%B4%EB%A8%B8%EB%A6%AC-%EB%82%A8%EC%84%B1-%EB%93%9C%EB%9D%BC%EC%9D%B4-%ED%95%8F-%EA%B8%B4%ED%8C%94-%EB%B2%84%EC%84%9C%ED%83%80%EC%9D%BC-%ED%83%91-RAjLBH2U/FB8586-110,나이키 프라이머리,탑 & 티셔츠,긴팔,69000
56,https://www.nike.com/kr/t/%EC%A1%B0%EB%8D%98-%EB%82%A8%EC%84%B1-%ED%81%AC%EB%A3%A8%EB%84%A5-%ED%8B%B0%EC%85%94%EC%B8%A0-VLdIyvtF/HQ2666-045,조던,탑 & 티셔츠,반팔,69000


In [57]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd

def open_link(b):
    # Dropdown에서 선택된 제품의 index
    selected_item_index = link_widget.value

    # 선택된 제품명 찾기
    selected_product = filtered_items['제품명'].iloc[selected_item_index]

    # 해당 인덱스를 통해 링크 가져오기
    selected_link = filtered_items['page_url'].iloc[selected_item_index]

    clear_output(wait=True)
    display(link_widget, open_button)

    # HTML 형식으로 클릭 가능한 링크 표시
    display(HTML(f'<a href="{selected_link}" target="_blank">링크 열기: {selected_product}</a>'))


# Dropdown 위젯 생성 (제품명 선택)
link_widget = widgets.Dropdown(
    options = [(f"{idx}: {row['제품명']}", idx) for idx, row in recommended_items.iterrows()],
    description="링크 선택: "
)

# 버튼 위젯 생성
open_button = widgets.Button(
    description="링크 열기"
)

# 버튼 클릭 시 open_link 함수 실행
open_button.on_click(open_link)

# UI와 출력 함께 표시
display(link_widget, open_button)


Dropdown(description='링크 선택: ', index=1, options=(('15: 에어 조던 워드마크 85', 15), ('11: 에어 조던 워드마크 85', 11), ('1: 조…

Button(description='링크 열기', style=ButtonStyle())